## combining individual coverage data into a mean-normalized coverage

In [5]:
ls /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov

001_A.chr7.bedCov     018_B.chr7.bedCov     037_A.chr7.bedCov
001_B.chr7.bedCov     020_A.chr7.bedCov     037_B.chr7.bedCov
002_A.chr7.bedCov     020_B.chr7.bedCov     038_A.chr7.bedCov
002_B.chr7.bedCov     021_A.chr7.bedCov     038_B.chr7.bedCov
003_A.chr7.bedCov     021_B.chr7.bedCov     039_A.chr7.bedCov
003_B.chr7.bedCov     022_A.chr7.bedCov     039_B.chr7.bedCov
004_A.chr7.bedCov     022_B.chr7.bedCov     040_A.chr7.bedCov
004_B.chr7.bedCov     023_A.chr7.bedCov     040_B.chr7.bedCov
005_A.chr7.bedCov     023_B.chr7.bedCov     041_A.chr7.bedCov
005_B.chr7.bedCov     024_A.chr7.bedCov     041_B.chr7.bedCov
006_A.chr7.bedCov     024_B.chr7.bedCov     042_A.chr7.bedCov
006_B.chr7.bedCov     025_A.chr7.bedCov     042_B.chr7.bedCov
007_A.chr7.bedCov     025_B.chr7.bedCov     043_A.chr7.bedCov
007_B.chr7.bedCov     026_A.chr7.bedCov     043_B.chr7.bedCov
008_A.chr7.bedCov     026_B.chr7.bedCov     044_A.chr7.bedCov
008_B.chr7.bedCov     028_A.chr7.bedCov     044_B.chr7.bedCov
009_A.ch

## Plan:

+ for the PanelofNormals, create a coverage file for each PON bam and each chromosome
+ Pon bams should optimally not contain CNV or there should be a lot of them to reduce the std
+ put all the files into a big matrix and normalize coverages and produce an average coverage (+ std) for the exonic space
+ compare the tumor samples against that PONcoverage to get differences in CNV

### have to look at the Kenichi-CNV-graphs to identify the samples from FDAH1 without CNVs in the normals
+ [1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [3]:
# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
tooldata = f"{home}/Dropbox/Icke/Work/somVar/tooldata"
shell_path = "../shell"
static_path = f"{home}/Dropbox/Icke/Work/static"
bed_path = f"{static_path}/bed_files/SureSelect/hg38"

cnvdata = f"{tooldata}/myCNVdata/"
bedCov_path = f"{cnvdata}/bedCov"
output_path = f"{cnvdata}/output"

In [4]:
test = pd.read_csv(f'{bedCov_path}/001_B.chr7.bedCov', sep='\t').set_index('#')
test[:13]

,Chr,Pos,Coverage
#,,,
1,chr7,19800,342.52
2,chr7,19850,455.40
3,chr7,19900,408.96
4,chr7,19950,251.77
5,chr7,20850,355.05
6,chr7,20900,415.78
7,chr7,20950,346.88
8,chr7,31150,25.74
9,chr7,31200,33.82


### get all the normal samples (without CNV) into df for normalization and averaging

In [8]:
import os
path = bedCov
samples = [1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
cov_dfs = []
for sample in [f"{str(s).zfill(3)}_B" for s in samples]:
    file = os.path.join(path, f"{sample}.chr7.bedCov")
    print(f"Reading {sample} from {file}.")
    df = pd.read_csv(file, sep='\t', usecols=['#', 'Coverage']).set_index('#').rename(columns={'Coverage':sample})
    cov_dfs.append(df)
cov_df = pd.concat(cov_dfs, axis=1).fillna(0)
cov_df
# the set_index('#') effectively deletes the chromosome info and creates an index of length rollingWindow / 2
# maybe that's not a good idea (better use an Exomic coordinate system)
cov_tidy = cov_df.stack().reset_index().set_index('#').rename(columns={'level_1':'sample', 0:'Coverage'})
cov_tidy.to_csv(f"{output_path}/all_tidy.bedCov", sep='\t')

Reading 001_B from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov/001_B.chr7.bedCov.
Reading 002_B from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov/002_B.chr7.bedCov.
Reading 004_B from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov/004_B.chr7.bedCov.
Reading 006_B from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov/006_B.chr7.bedCov.
Reading 007_B from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov/007_B.chr7.bedCov.
Reading 008_B from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov/008_B.chr7.bedCov.
Reading 009_B from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov/009_B.chr7.bedCov.
Reading 010_B from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov/010_B.chr7.bedCov.
Reading 011_B from /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/bedCov/011_B.chr7.bedCov.
Reading 01

,001_B,002_B,004_B,006_B,007_B,008_B,009_B,010_B,011_B,012_B,013_B,014_B,015_B,016_B
#,,,,,,,,,,,,,,
1,342.52,342.04,486.00,685.55,82.06,343.06,483.96,498.93,430.21,563.28,521.82,332.35,463.18,491.97
2,455.40,481.48,634.28,1013.19,135.85,461.33,690.60,702.79,563.28,794.89,730.47,485.60,670.10,752.03
3,408.96,478.19,589.93,969.48,145.17,424.46,647.08,685.54,505.18,761.39,699.50,481.77,643.44,724.96
4,251.77,333.30,386.88,598.11,102.55,269.62,397.13,465.10,322.64,494.59,442.17,322.86,408.79,424.09
5,355.05,328.92,531.10,472.55,80.78,284.31,428.72,372.95,320.48,384.37,394.53,408.62,437.75,497.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48639,0.00,0.00,409.71,0.00,0.00,196.13,0.00,196.00,0.00,0.00,0.00,297.01,0.00,147.97
48640,0.00,0.00,316.89,0.00,0.00,134.55,0.00,150.77,0.00,0.00,0.00,251.15,0.00,111.89
48641,0.00,0.00,205.16,0.00,0.00,0.00,0.00,87.67,0.00,0.00,0.00,164.46,0.00,98.85


In [10]:
# normalize sample coverages to an average coverage of 250
norm_cov = cov_df / cov_df.mean() * 250
norm_cov
norm_cov.to_csv(f"{bedCov}/all_norm.bedCov", sep='\t')

normtidy = norm_cov.stack().reset_index().set_index('#').rename(columns={'level_1':'sample', 0:'Coverage'})
normtidy.to_csv(f"{bedCov}/all_norm_tidy.bedCov", sep='\t')
normtidy

,001_B,002_B,004_B,006_B,007_B,008_B,009_B,010_B,011_B,012_B,013_B,014_B,015_B,016_B
#,,,,,,,,,,,,,,
1,167.627918,165.416396,192.552439,280.156273,141.359222,172.580876,248.173834,208.352808,210.061560,254.249411,236.069283,142.357876,217.015463,238.012396
2,222.870938,232.851966,251.300742,414.049354,234.019624,232.078166,354.138461,293.484598,275.036553,358.791923,330.461709,208.000556,313.964466,363.828002
3,200.143388,231.260866,233.729342,396.186863,250.074559,213.530224,331.821482,286.281010,246.667671,343.670926,316.451005,206.360024,301.473357,350.731684
4,123.215231,161.189583,153.281250,244.423118,176.655962,135.635912,203.647563,194.225425,157.537625,223.244597,200.035941,138.292956,191.531912,205.172423
5,173.760050,159.071340,210.420988,193.111876,139.154253,143.025911,219.846859,155.743651,156.482947,173.494259,178.483796,175.027156,205.100649,240.470072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48639,0.000000,0.000000,162.326460,0.000000,0.000000,98.665794,0.000000,81.849459,0.000000,0.000000,0.000000,127.220439,0.000000,71.587077
48640,0.000000,0.000000,125.551322,0.000000,0.000000,67.687159,0.000000,62.961443,0.000000,0.000000,0.000000,107.576894,0.000000,54.131770
48641,0.000000,0.000000,81.284071,0.000000,0.000000,0.000000,0.000000,36.610929,0.000000,0.000000,0.000000,70.444340,0.000000,47.823089


,sample,Coverage
#,,
1,001_B,167.627918
1,002_B,165.416396
1,004_B,192.552439
1,006_B,280.156273
1,007_B,141.359222
...,...,...
48643,012_B,0.000000
48643,013_B,0.000000
48643,014_B,0.000000


### avg_cov has the coverage of the "panel of normals"

In [12]:
avg_cov = pd.DataFrame(index=norm_cov.index)
avg_cov['meanCov'] = norm_cov.mean(axis=1)
avg_cov['std'] = norm_cov.std(axis=1)
avg_cov['std'].mean()
avg_cov

108.2299768722271

,meanCov,std
#,,
1,205.284697,43.249567
2,291.776933,64.789996
3,279.170171,61.894831
4,179.149250,35.578266
5,180.228129,29.992631
...,...,...
48639,38.689231,57.604694
48640,29.850613,45.010288
48641,16.868745,29.369201


### get all tumor samples and normalize

In [6]:
samplesA = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,21]
cov_dfs = []
for sample in [f"{str(s).zfill(3)}_A" for s in samplesA]:
    file = os.path.join(path, f"{sample}.chr7.bedCov")
    print(f"Reading {sample} from {file}.")
    df = pd.read_csv(file, sep='\t', usecols=['#', 'Coverage']).set_index('#').rename(columns={'Coverage':sample})
    cov_dfs.append(df)
covA_df = pd.concat(cov_dfs, axis=1).fillna(0)
covA_df
covA_tidy = covA_df.stack().reset_index().set_index('#').rename(columns={'level_1':'sample', 0:'Coverage'})
covA_tidy.to_csv(f"{bedCov}/allA_tidy.bedCov", sep='\t')

Reading 001_A from ../bedCov/001_A.chr7.bedCov.
Reading 002_A from ../bedCov/002_A.chr7.bedCov.
Reading 003_A from ../bedCov/003_A.chr7.bedCov.
Reading 004_A from ../bedCov/004_A.chr7.bedCov.
Reading 005_A from ../bedCov/005_A.chr7.bedCov.
Reading 006_A from ../bedCov/006_A.chr7.bedCov.
Reading 007_A from ../bedCov/007_A.chr7.bedCov.
Reading 008_A from ../bedCov/008_A.chr7.bedCov.
Reading 009_A from ../bedCov/009_A.chr7.bedCov.
Reading 010_A from ../bedCov/010_A.chr7.bedCov.
Reading 011_A from ../bedCov/011_A.chr7.bedCov.
Reading 012_A from ../bedCov/012_A.chr7.bedCov.
Reading 013_A from ../bedCov/013_A.chr7.bedCov.
Reading 014_A from ../bedCov/014_A.chr7.bedCov.
Reading 015_A from ../bedCov/015_A.chr7.bedCov.
Reading 016_A from ../bedCov/016_A.chr7.bedCov.
Reading 021_A from ../bedCov/021_A.chr7.bedCov.


,001_A,002_A,003_A,004_A,005_A,006_A,007_A,008_A,009_A,010_A,011_A,012_A,013_A,014_A,015_A,016_A,021_A
#,,,,,,,,,,,,,,,,,
1,349.05,498.71,285.36,439.88,433.74,727.27,255.64,349.36,612.06,625.83,523.42,789.77,620.74,412.63,387.80,396.63,188.50
2,502.48,688.33,385.41,564.85,664.15,1121.34,361.51,452.91,906.54,847.05,720.88,1061.39,861.73,599.49,566.29,551.71,268.71
3,485.78,658.99,369.79,521.23,654.36,1108.93,351.49,402.40,888.03,781.83,682.56,1000.91,819.22,578.60,526.01,521.48,257.41
4,316.60,439.89,260.17,350.51,406.52,698.48,233.04,242.52,564.43,496.54,436.13,655.17,520.61,369.81,309.13,321.66,165.32
5,417.00,393.06,498.26,437.26,407.54,444.09,315.55,271.30,304.76,449.70,423.08,495.19,449.71,449.15,562.17,377.56,168.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48634,0.00,158.91,308.61,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,94.52,0.00,0.00,0.00,222.13,0.00
48635,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,164.12,0.00
48636,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,118.18,0.00


In [7]:
normA_cov = covA_df / covA_df.mean() * 250
normA_cov

,001_A,002_A,003_A,004_A,005_A,006_A,007_A,008_A,009_A,010_A,011_A,012_A,013_A,014_A,015_A,016_A,021_A
#,,,,,,,,,,,,,,,,,
1,176.619490,199.726533,127.421889,343.280611,230.266648,306.333443,151.189863,243.525640,331.952951,324.474278,251.092754,311.238477,291.640626,183.417256,179.668089,240.427761,240.374025
2,254.255154,275.666749,172.097246,440.806704,352.588173,472.319692,213.803189,315.706428,491.665242,439.170282,345.817403,418.280521,404.864317,266.477984,262.362666,334.433603,342.657317
3,245.804945,263.916480,165.122443,406.765829,347.390795,467.092475,207.877190,280.497818,481.626277,405.355648,327.434700,394.446110,384.891957,257.192216,243.700906,316.108889,328.247627
4,160.199773,176.169927,116.173790,273.536616,215.815921,294.206805,137.823837,169.051518,306.120649,257.441251,209.218377,258.194301,244.596814,164.383431,143.220207,194.982713,210.815033
5,211.002227,157.415153,222.488191,341.235973,216.357425,187.055177,186.621660,189.112967,165.287686,233.156101,202.958088,195.148184,211.286055,199.650681,260.453866,228.867976,214.933909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48634,0.000000,63.641281,137.803718,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37.249150,0.000000,0.000000,0.000000,134.649972,0.000000
48635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.485677,0.000000
48636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,71.637932,0.000000


### get the normalized difCov for every sample

In [14]:
difCov = pd.DataFrame(index=normA_cov.index)

In [31]:
for col in normA_cov.columns:
    difCov[col] = normA_cov[col] / avg_cov['meanCov'] * 2
    difCov[col] = difCov[col].rolling(100).mean()

In [32]:
difCov
difCov_tidy = difCov.stack().reset_index().set_index('#').rename(columns={'level_1':'sample', 0:'Coverage'})
difCov_tidy.to_csv("../bedCov/divCov_tidy.bedCov", sep='\t')

,001_A,002_A,003_A,004_A,005_A,006_A,007_A,008_A,009_A,010_A,011_A,012_A,013_A,014_A,015_A,016_A,021_A
#,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48634,1.367288,2.045857,2.882795,2.009125,0.312594,1.703389,3.137690,2.137259,1.370184,1.838662,1.750852,2.083453,1.233746,2.066507,1.699713,2.354668,1.509845
48635,1.357475,2.029291,2.861164,1.991200,0.292962,1.684336,3.093482,2.111089,1.345403,1.813750,1.725805,2.064383,1.202030,2.037433,1.682546,2.358861,1.497673
48636,1.344678,2.002462,2.824877,1.979764,0.279512,1.656052,3.042178,2.093662,1.309906,1.796468,1.706297,2.032133,1.173202,2.002712,1.661941,2.348511,1.485767


In [26]:
difCov['001_A'].rolling(1000).mean()

#
1             NaN
2             NaN
3             NaN
4             NaN
5             NaN
           ...   
48634    1.423354
48635    1.415789
48636    1.406715
48637    1.396792
48638    1.389984
Name: 001_A, Length: 48638, dtype: float64